# Future Sales Prediction
 
 
 
This is a challenging time-series dataset consisting of daily sales data, 
kindly provided by one of the largest Russian software firms - 1C Company. 
We need to predict total sales for every product and store in the next month.
 
This dataset has been acquired from the kaggel platform 
(https://www.kaggle.com/c/competitive-data-science-predict-future-sales/overview/evaluation)

 
 



###### The objective requires us to predict sales for the next month at a store-item combination.
  *  First We can predict Sales for whole company
  *  Then we can plan store wise prediction 
  *  Item-wise prediction 
  *  We can decide the popular store
  *  Different 
 
 It may help customer to take a call on business expansion needs to be done on which area

## Data Preparation, Exploratory Analysis

Following steps can be performed for data preparation, analysis and modelling to get the best predictions. The list might not be complete but have tried to list down as much as possible.
###### STATISTICALLY EXPLORING DATA - A CHECK LIST
* Check data dimensions, Rows and Columns
* Columns Names
* Datatype(s) and Data Info
* Unique Values per Column (to decide whether to make column Categorical?)
###### CLEANING DATA
* Look for any Missing Data
* Can identify observations on Missing Data
* Graph the representations of Missing Data
* Decide whether to populate or to remove Missing Data 
* Identify the possible impact of it while Modelling

##### Some questions to guide the initial exploration:

* Data shape, missings, first rows,
* What are the entropy of each column
* What are the principal shops?
* What are the distributions of items price and total items sold by each item;
* What are the more sold items and which are their categories;
* What's the range of date sales;
* How many items were sold each day. Could we see any peak in christmas, valentine's day or another special day.
* Crossing some of this features
And a lot of more questions that probably will raise through the exploration.
Further we need to analyze the archived data statistically 
We need to fit the best Machine Learning model for further prediction

## File descriptions

**sales_train.csv** - the training set. Daily historical data from January 2013 to October 2015.

**test.csv** - the test set. You need to forecast the sales for these shops and products for November 2015.

**sample_submission.csv** - a sample submission file in the correct format.

**items.csv** - supplemental information about the items/products.

**item_categories.csv**  - supplemental information about the items categories.

**shops.csv** - supplemental information about the shops.

## Data fields

**ID** - an Id that represents a (Shop, Item) tuple within the test set

**shop_id** - unique identifier of a shop

**item_id** - unique identifier of a product

**item_category_id** - unique identifier of item category

**item_cnt_day** - number of products sold. You are predicting a monthly amount of this measure

**item_price** - current price of an item

**date** - date in format dd/mm/yyyy

**date_block_num** - a consecutive month number, used for convenience. January 2013 is 0, February 2013 is 1,..., October 2015 is 33

**item_name** - name of item

**shop_name** - name of shop

**item_category_name** - name of item category

In [4]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
from scipy import stats
import random
import gc

### Importing the Datasets


In [5]:

df_train = pd.read_csv('sales_train.csv')

df_categories = pd.read_csv("item_categories.csv")

df_items = pd.read_csv("items.csv")

df_shops = pd.read_csv("shops.csv")

df_test = pd.read_csv("test.csv")


In [6]:
gc.collect()

67

## Findin the shape of the data & checking for any missing values

In [7]:
df_test.head()

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


In [8]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 214200 entries, 0 to 214199
Data columns (total 3 columns):
ID         214200 non-null int64
shop_id    214200 non-null int64
item_id    214200 non-null int64
dtypes: int64(3)
memory usage: 4.9 MB


In [9]:
df_train.head(10)

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0
5,10.01.2013,0,25,2564,349.00,1.0
6,02.01.2013,0,25,2565,549.00,1.0
7,04.01.2013,0,25,2572,239.00,1.0
8,11.01.2013,0,25,2572,299.00,1.0
9,03.01.2013,0,25,2573,299.00,3.0


In [10]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2935849 entries, 0 to 2935848
Data columns (total 6 columns):
date              object
date_block_num    int64
shop_id           int64
item_id           int64
item_price        float64
item_cnt_day      float64
dtypes: float64(2), int64(3), object(1)
memory usage: 134.4+ MB


In [11]:
df_shops.head()

,shop_name,shop_id
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1
2,"Адыгея ТЦ ""Мега""",2
3,"Балашиха ТРК ""Октябрь-Киномир""",3
4,"Волжский ТЦ ""Волга Молл""",4


In [12]:
df_shops.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 2 columns):
shop_name    60 non-null object
shop_id      60 non-null int64
dtypes: int64(1), object(1)
memory usage: 1.1+ KB


In [13]:
df_items.head()

,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40


In [14]:
df_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22170 entries, 0 to 22169
Data columns (total 3 columns):
item_name           22170 non-null object
item_id             22170 non-null int64
item_category_id    22170 non-null int64
dtypes: int64(2), object(1)
memory usage: 519.7+ KB


In [15]:
df_categories.head()

,item_category_name,item_category_id
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4


In [16]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2935849 entries, 0 to 2935848
Data columns (total 6 columns):
date              object
date_block_num    int64
shop_id           int64
item_id           int64
item_price        float64
item_cnt_day      float64
dtypes: float64(2), int64(3), object(1)
memory usage: 134.4+ MB


#### There is no missing data in the given dataset 
* If any missing values were foun we might need to dorp the column, or forwardfill or backward fill depending on the situation 

###### check if all shop and items in the test set is also in the training set.

In [17]:
test_shops = df_test.shop_id.unique()
train = df_train[df_train.shop_id.isin(test_shops)]
test_items = df_test.item_id.unique()
train1 = df_train[df_train.item_id.isin(test_items)]
train1

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.0,1.0
10,03.01.2013,0,25,2574,399.0,2.0
11,05.01.2013,0,25,2574,399.0,1.0
12,07.01.2013,0,25,2574,399.0,1.0
13,08.01.2013,0,25,2574,399.0,2.0
...,...,...,...,...,...,...
2935844,10.10.2015,33,25,7409,299.0,1.0
2935845,09.10.2015,33,25,7460,299.0,1.0
2935846,14.10.2015,33,25,7459,349.0,1.0
2935847,22.10.2015,33,25,7440,299.0,1.0


In [18]:
train

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0
...,...,...,...,...,...,...
2935844,10.10.2015,33,25,7409,299.00,1.0
2935845,09.10.2015,33,25,7460,299.00,1.0
2935846,14.10.2015,33,25,7459,349.00,1.0
2935847,22.10.2015,33,25,7440,299.00,1.0


### Joining the tables to our train dataset

In [24]:
df_train_m = pd.merge(df_train, df_items, on='item_id', how='inner')

df_train_m = pd.merge(df_train_m, df_categories, on='item_category_id', how='inner')
df_train_m = pd.merge(df_train_m, df_shops, on='shop_id', how='inner')

df_test_m = pd.merge(df_test, df_items, on='item_id', how='inner')
df_test_m = pd.merge(df_test_m, df_categories, on='item_category_id', how='inner')
df_test_m = pd.merge(df_test_m, df_shops, on='shop_id', how='inner')

### Look at the merge train dataset 

In [25]:
df_train_m.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_name,item_category_id,item_category_name,shop_name
0,02.01.2013,0,59,22154,999.0,1.0,ЯВЛЕНИЕ 2012 (BD),37,Кино - Blu-Ray,"Ярославль ТЦ ""Альтаир"""
1,26.04.2013,3,59,944,150.0,1.0,2012 (BD),37,Кино - Blu-Ray,"Ярославль ТЦ ""Альтаир"""
2,26.06.2013,5,59,944,199.5,1.0,2012 (BD),37,Кино - Blu-Ray,"Ярославль ТЦ ""Альтаир"""
3,20.07.2013,6,59,944,199.5,1.0,2012 (BD),37,Кино - Blu-Ray,"Ярославль ТЦ ""Альтаир"""
4,14.09.2013,8,59,944,299.0,2.0,2012 (BD),37,Кино - Blu-Ray,"Ярославль ТЦ ""Альтаир"""


In [26]:
df_train_m.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2935849 entries, 0 to 2935848
Data columns (total 10 columns):
date                  object
date_block_num        int64
shop_id               int64
item_id               int64
item_price            float64
item_cnt_day          float64
item_name             object
item_category_id      int64
item_category_name    object
shop_name             object
dtypes: float64(2), int64(4), object(4)
memory usage: 246.4+ MB


In [29]:
df_test_m.head(6)

,ID,shop_id,item_id,item_name,item_category_id,item_category_name,shop_name
0,0,5,5037,"NHL 15 [PS3, русские субтитры]",19,Игры - PS3,"Вологда ТРЦ ""Мармелад"""
1,2,5,5233,"Need for Speed Rivals (Essentials) [PS3, русск...",19,Игры - PS3,"Вологда ТРЦ ""Мармелад"""
2,15,5,4872,"Minecraft. Playstation 3 Edition [PS3, русская...",19,Игры - PS3,"Вологда ТРЦ ""Мармелад"""
3,21,5,5002,"NBA 2K16 [PS3, русская документация]",19,Игры - PS3,"Вологда ТРЦ ""Мармелад"""
4,23,5,5814,"Plants vs. Zombies Garden Warfare [PS3, русска...",19,Игры - PS3,"Вологда ТРЦ ""Мармелад"""
5,56,5,3341,"FIFA 15 [PS3, русская версия]",19,Игры - PS3,"Вологда ТРЦ ""Мармелад"""


In [28]:
df_test_m.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 214200 entries, 0 to 214199
Data columns (total 7 columns):
ID                    214200 non-null int64
shop_id               214200 non-null int64
item_id               214200 non-null int64
item_name             214200 non-null object
item_category_id      214200 non-null int64
item_category_name    214200 non-null object
shop_name             214200 non-null object
dtypes: int64(4), object(3)
memory usage: 13.1+ MB


## End of Data Wrangling and saving the merged dataferame further 



In [30]:
df_test_m.to_csv('merged_test_data.csv')

In [31]:
df_train_m.to_csv('merged_tain_data.csv')